In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('StructureAPI').config('spark.some.config.option', 'some-value').getOrCreate()

In [2]:
from pyspark.sql.functions import col, column, expr
from pyspark.sql import functions as f

In [3]:
df = spark.read.json('movies.json')

In [4]:
df.show()

+----------------+--------------------+--------------------+----+
|            cast|              genres|               title|year|
+----------------+--------------------+--------------------+----+
|              []|                  []|After Dark in Cen...|1900|
|              []|                  []|Boarding School G...|1900|
|              []|                  []|Buffalo Bill's Wi...|1900|
|              []|                  []|              Caught|1900|
|              []|                  []|Clowns Spinning Hats|1900|
|              []|[Short, Documentary]|Capture of Boer B...|1900|
|              []|                  []|The Enchanted Dra...|1900|
|   [Paul Boyton]|                  []|   Feeding Sea Lions|1900|
|              []|            [Comedy]|How to Make a Fat...|1900|
|              []|                  []|     New Life Rescue|1900|
|              []|                  []|    New Morning Bath|1900|
|              []|                  []|Searching Ruins o...|1900|
|         

In [5]:
df.printSchema()

root
 |-- cast: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [6]:
# 1. Show the number of distinct fimls in the dataset
df.distinct().count()

28789

In [7]:
# 2. Count the number of films released during the years 2012 and 2015 (included)
df.filter((col('year') >= 2012) & (col('year') <= 2015)).count()

1015

In [10]:
# 3. Show the year in which the number of movies released is highest. Only one highest year is expected in the output.
df.groupBy('year').count().sort('count', ascending=False).limit(1).show()

+----+-----+
|year|count|
+----+-----+
|1919|  634|
+----+-----+



In [15]:
# 4. Show the list of movies such that for each film, 
# the number of actors/actresses is at least five, and the number of genres it belongs to is at most two genres.
df.filter((f.size('cast') >= 5) & (f.size('genres') <= 2)).show()

+--------------------+----------------+--------------------+----+
|                cast|          genres|               title|year|
+--------------------+----------------+--------------------+----+
|[Earle Foxe, Alie...|         [Drama]|  A Desperate Chance|1913|
|[Charlotte Burton...|         [Drama]|    The Archeologist|1914|
|[Charlotte Burton...|         [Drama]|At the Potter's W...|1914|
|[Herbert Tracey, ...|        [Comedy]|    Back to the Farm|1914|
|[Charlotte Burton...|              []|    The Beggar Child|1914|
|[William Garwood,...|              []|       Billy's Rival|1914|
|[B. Reeves Eason,...|         [Drama]| Break, Break, Break|1914|
|[Charlotte Burton...|              []|       The Butterfly|1914|
|[Charlotte Burton...|       [Western]|Calamity Anne's L...|1914|
|[Charlie Chaplin,...|        [Comedy]|    The Star Boarder|1914|
|[Sydney Ayres, Ja...|              []|A Story of Little...|1914|
|[Sydney Ayres, Pe...|              []|The Story of the ...|1914|
|[Charlott